Instalando dependências do *Spark* (utilizar apenas no google colab)

In [3]:
# !apt-get update -qq
# !apt-get install openjdk-8-jdk-headless -qq > /dev/null
# !wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz
# !tar xf spark-3.1.2-bin-hadoop2.7.tgz
# !pip install -q findspark

# Importando OS
# import os
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

In [4]:
import findspark
findspark.init()

Importando *SparkSession*

In [5]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master('local[*]') \
    .appName("Iniciando com Spark") \
    .getOrCreate()

In [6]:
spark

Importando bibliotecas

In [7]:
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns

In [8]:
sns.set()

# Carregando os dados no *dataframe*

In [110]:
#Use this path in windows
csv_path_windows = r"C:\Users\paulo\PycharmProjects\ETL-CVM\data\cvm_fi_2000_2022.csv"
#Use this path in colab
colab_path = '/content/drive/MyDrive/cvm_2000_2022.csv'

df = spark.read.csv(
    csv_path_windows,
    header= True, inferSchema= True
)

In [111]:
df.show(5)

+---+--------+------------------+-------------------+--------+---------+-------------+---------+--------+--------+
|_c0|TP_FUNDO|        CNPJ_FUNDO|          DT_COMPTC|VL_TOTAL| VL_QUOTA|VL_PATRIM_LIQ|CAPTC_DIA|RESG_DIA|NR_COTST|
+---+--------+------------------+-------------------+--------+---------+-------------+---------+--------+--------+
|  0|   FITVM|01.465.738/0001-97|2000-01-03 00:00:00|    null|2.5972313|    2588000.0|  91000.0|     0.0|    null|
|  1|   FITVM|01.465.738/0001-97|2000-01-04 00:00:00|    null|2.4597298|    2543000.0|      0.0| 17000.0|    null|
|  2|   FITVM|01.465.738/0001-97|2000-01-05 00:00:00|    null|2.4035701|    2467000.0|  44000.0|  2000.0|    null|
|  3|   FITVM|01.465.738/0001-97|2000-01-06 00:00:00|    null|2.4648947|    2573000.0|  21000.0|     0.0|    null|
|  4|   FITVM|01.465.738/0001-97|2000-01-07 00:00:00|    null|2.5131104|    2646000.0|   3000.0|     0.0|    null|
+---+--------+------------------+-------------------+--------+---------+--------

## Renomeação da coluna `_c0`

In [112]:
df = df.withColumnRenamed('_c0', 'index')

In [113]:
df.show(5)

+-----+--------+------------------+-------------------+--------+---------+-------------+---------+--------+--------+
|index|TP_FUNDO|        CNPJ_FUNDO|          DT_COMPTC|VL_TOTAL| VL_QUOTA|VL_PATRIM_LIQ|CAPTC_DIA|RESG_DIA|NR_COTST|
+-----+--------+------------------+-------------------+--------+---------+-------------+---------+--------+--------+
|    0|   FITVM|01.465.738/0001-97|2000-01-03 00:00:00|    null|2.5972313|    2588000.0|  91000.0|     0.0|    null|
|    1|   FITVM|01.465.738/0001-97|2000-01-04 00:00:00|    null|2.4597298|    2543000.0|      0.0| 17000.0|    null|
|    2|   FITVM|01.465.738/0001-97|2000-01-05 00:00:00|    null|2.4035701|    2467000.0|  44000.0|  2000.0|    null|
|    3|   FITVM|01.465.738/0001-97|2000-01-06 00:00:00|    null|2.4648947|    2573000.0|  21000.0|     0.0|    null|
|    4|   FITVM|01.465.738/0001-97|2000-01-07 00:00:00|    null|2.5131104|    2646000.0|   3000.0|     0.0|    null|
+-----+--------+------------------+-------------------+--------+

## Dimensões do *dataframe*

In [114]:
print(f'Há no dataframe {len(df.columns)} colunas.')
print(f'E {df.select("TP_FUNDO").count()} linhas.')

Há no dataframe 10 colunas.
E 57626454 linhas.


## *Schema* do *dataframe*

In [115]:
df.printSchema()

root
 |-- index: integer (nullable = true)
 |-- TP_FUNDO: string (nullable = true)
 |-- CNPJ_FUNDO: string (nullable = true)
 |-- DT_COMPTC: timestamp (nullable = true)
 |-- VL_TOTAL: double (nullable = true)
 |-- VL_QUOTA: double (nullable = true)
 |-- VL_PATRIM_LIQ: double (nullable = true)
 |-- CAPTC_DIA: double (nullable = true)
 |-- RESG_DIA: double (nullable = true)
 |-- NR_COTST: double (nullable = true)



# Derivação de novas colunas a partir da coluna `DT_COMPTC`

### Colunas `MONTH` e `YEAR`

In [116]:
import pyspark.sql.functions as f

In [117]:
df = df.withColumn('MONTH', f.month('DT_COMPTC'))\
       .withColumn('YEAR', f.year('DT_COMPTC'))

In [118]:
df.show(2)

+-----+--------+------------------+-------------------+--------+---------+-------------+---------+--------+--------+-----+----+
|index|TP_FUNDO|        CNPJ_FUNDO|          DT_COMPTC|VL_TOTAL| VL_QUOTA|VL_PATRIM_LIQ|CAPTC_DIA|RESG_DIA|NR_COTST|MONTH|YEAR|
+-----+--------+------------------+-------------------+--------+---------+-------------+---------+--------+--------+-----+----+
|    0|   FITVM|01.465.738/0001-97|2000-01-03 00:00:00|    null|2.5972313|    2588000.0|  91000.0|     0.0|    null|    1|2000|
|    1|   FITVM|01.465.738/0001-97|2000-01-04 00:00:00|    null|2.4597298|    2543000.0|      0.0| 17000.0|    null|    1|2000|
+-----+--------+------------------+-------------------+--------+---------+-------------+---------+--------+--------+-----+----+
only showing top 2 rows



### Coluna `MONTH_OF_YEAR`

In [119]:
df = df.withColumn('MONTH_OF_YEAR', f.date_format(df['DT_COMPTC'], 'yyyy-MM'))

In [120]:
df.show(2)

+-----+--------+------------------+-------------------+--------+---------+-------------+---------+--------+--------+-----+----+-------------+
|index|TP_FUNDO|        CNPJ_FUNDO|          DT_COMPTC|VL_TOTAL| VL_QUOTA|VL_PATRIM_LIQ|CAPTC_DIA|RESG_DIA|NR_COTST|MONTH|YEAR|MONTH_OF_YEAR|
+-----+--------+------------------+-------------------+--------+---------+-------------+---------+--------+--------+-----+----+-------------+
|    0|   FITVM|01.465.738/0001-97|2000-01-03 00:00:00|    null|2.5972313|    2588000.0|  91000.0|     0.0|    null|    1|2000|      2000-01|
|    1|   FITVM|01.465.738/0001-97|2000-01-04 00:00:00|    null|2.4597298|    2543000.0|      0.0| 17000.0|    null|    1|2000|      2000-01|
+-----+--------+------------------+-------------------+--------+---------+-------------+---------+--------+--------+-----+----+-------------+
only showing top 2 rows



### Coluna `QUARTER_OF_YEAR`

In [121]:
df = df.withColumn('QUARTER', f.concat(f.year('DT_COMPTC'), f.quarter('DT_COMPTC')))

In [122]:
df.show(2)

+-----+--------+------------------+-------------------+--------+---------+-------------+---------+--------+--------+-----+----+-------------+-------+
|index|TP_FUNDO|        CNPJ_FUNDO|          DT_COMPTC|VL_TOTAL| VL_QUOTA|VL_PATRIM_LIQ|CAPTC_DIA|RESG_DIA|NR_COTST|MONTH|YEAR|MONTH_OF_YEAR|QUARTER|
+-----+--------+------------------+-------------------+--------+---------+-------------+---------+--------+--------+-----+----+-------------+-------+
|    0|   FITVM|01.465.738/0001-97|2000-01-03 00:00:00|    null|2.5972313|    2588000.0|  91000.0|     0.0|    null|    1|2000|      2000-01|  20001|
|    1|   FITVM|01.465.738/0001-97|2000-01-04 00:00:00|    null|2.4597298|    2543000.0|      0.0| 17000.0|    null|    1|2000|      2000-01|  20001|
+-----+--------+------------------+-------------------+--------+---------+-------------+---------+--------+--------+-----+----+-------------+-------+
only showing top 2 rows



# Análise exploratória

## Quantidade de dados faltantes

In [123]:
df.select([f.count(f.when(f.isnull(c), c)).alias(c) for c in df.columns]).show()

+-----+--------+----------+---------+--------+--------+-------------+---------+--------+--------+-----+----+-------------+-------+
|index|TP_FUNDO|CNPJ_FUNDO|DT_COMPTC|VL_TOTAL|VL_QUOTA|VL_PATRIM_LIQ|CAPTC_DIA|RESG_DIA|NR_COTST|MONTH|YEAR|MONTH_OF_YEAR|QUARTER|
+-----+--------+----------+---------+--------+--------+-------------+---------+--------+--------+-----+----+-------------+-------+
|    0|40860852|         0|        0|  366012|       0|            0|        0|       0|  365408|    0|   0|            0|      0|
+-----+--------+----------+---------+--------+--------+-------------+---------+--------+--------+-----+----+-------------+-------+



In [124]:
df.agg(*[f.mean(f.when(f.col(c).isNull(), 1).otherwise(0)).alias(c) for c in df.columns]).show()

+-----+------------------+----------+---------+--------------------+--------+-------------+---------+--------+-------------------+-----+----+-------------+-------+
|index|          TP_FUNDO|CNPJ_FUNDO|DT_COMPTC|            VL_TOTAL|VL_QUOTA|VL_PATRIM_LIQ|CAPTC_DIA|RESG_DIA|           NR_COTST|MONTH|YEAR|MONTH_OF_YEAR|QUARTER|
+-----+------------------+----------+---------+--------------------+--------+-------------+---------+--------+-------------------+-----+----+-------------+-------+
|  0.0|0.7090641391885747|       0.0|      0.0|0.006351457960609549|     0.0|          0.0|      0.0|     0.0|0.00634097666325261|  0.0| 0.0|          0.0|    0.0|
+-----+------------------+----------+---------+--------------------+--------+-------------+---------+--------+-------------------+-----+----+-------------+-------+



Há uma quantidade bastante expressiva de dados faltantes na coluna `TP_FUNDO`. Os dados faltantes respondem por, aproximadamente, 71% dos dados da coluna.

## Coluna `TP_FUNDO`

In [125]:
count_tp_fundo = df.groupBy('TP_FUNDO').count().toPandas().sort_values('count', ascending= False)

In [126]:
count_tp_fundo

,TP_FUNDO,count
1,None,40860852
0,FI,15216503
4,FIF,500215
9,FACFIF,492620
8,FITVM,349153
3,FMP-FGTS,107997
6,FIC-FITVM,67022
7,FIEX,13624
11,FAPI,9369
10,FMP-FGTS CL,8226


## Coluna `VL_TOTAL`

### Estatísticas descritivas

In [127]:
df.describe('VL_TOTAL').show(truncate= False)

+-------+---------------------+
|summary|VL_TOTAL             |
+-------+---------------------+
|count  |57260442             |
|mean   |3.528318584928253E8  |
|stddev |2.9904451762515408E10|
|min    |-5.24478822941659E12 |
|max    |2.15700309082785E14  |
+-------+---------------------+



### Média diária do VL_TOTAL

In [128]:
avg_vl_total_per_date = df.groupBy('DT_COMPTC').agg(f.avg('VL_TOTAL')).sort('DT_COMPTC').toPandas()

C:\Spark\spark-3.3.1-bin-hadoop3\python\pyspark\sql\pandas\conversion.py:248: FutureWarning:

Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead



Para identificar a existências de valores discrepantes plota-se um gráfico para observar as médias do `VL_TOTAL` em cada data:

In [ ]:
fig = px.line(
    data_frame= avg_vl_total_per_date,
    x= 'DT_COMPTC',
    y= 'avg(VL_TOTAL)',
    markers= True,
    title= 'Média diária da variável VL_TOTAL'
)
fig.update_layout(title= {'x': 0.5}, xaxis_title= 'Dia', yaxis_title= 'Média do VL_TOTAL')
fig.show()

### *Boxplot* da média diária do `VL_TOTAL`

In [ ]:
fig = px.box(avg_vl_total_per_date['avg(VL_TOTAL)'], title= 'Boxplot da variável VL_TOTAL', orientation= 'h')
fig.update_layout(title= {'x': 0.5}, yaxis_title= '', xaxis_title= 'Valor')
fig.show()

### Remoção de *outliers* da variável `VL_TOTAL`

In [ ]:
Q1 = df.approxQuantile('VL_TOTAL', [0.25], 0.01)[0] # os parâmetros para o método approxQuantile são, respectivamente: coluna, quantil e erro.
Q3 = df.approxQuantile('VL_TOTAL', [0.75], 0.01)[0] # é retornada uma lista com o valor, por isso utiliza-se o [0], para obter apenas o valor.
IIQ = Q3 - Q1

In [ ]:
inferior = Q1 - (1.5 * IIQ)
superior = Q3 + (1.5 * IIQ)

In [ ]:
vl_total_without_outliers = df.where((df['VL_TOTAL'] >= inferior) & (df['VL_TOTAL'] <= superior) & (df['VL_TOTAL'] != 0)) # removem-se os dados onde a média
# é zero.

In [ ]:
f'Número de linhas: {vl_total_without_outliers.count()}'


### Análises após a remoção dos *outliers*

#### Estatísticas descritivas

In [ ]:
vl_total_without_outliers.describe('VL_TOTAL').show()

#### Medidas de tendência central para variável `VL_TOTAL`

##### Média diária

In [ ]:
avg_vl_total_per_date = vl_total_without_outliers.groupBy('DT_COMPTC').agg(f.avg('VL_TOTAL')).sort('DT_COMPTC').toPandas()

In [ ]:
fig = px.line(
    data_frame= avg_vl_total_per_date,
    x= 'DT_COMPTC',
    y= 'avg(VL_TOTAL)',
    markers= True,
    title= 'Média diária do VL_TOTAL'
)
fig.update_layout(title= {'x': 0.5}, yaxis_title= 'Média do VL_TOTAL', xaxis_title= 'Dia')
fig.show()


##### Média mensal

In [ ]:
avg_vl_total_per_month = vl_total_without_outliers.groupby('MONTH_OF_YEAR').agg(f.avg('VL_TOTAL')).orderBy('MONTH_OF_YEAR').toPandas()

In [ ]:
fig = px.line(
    data_frame= avg_vl_total_per_month,
    x= 'MONTH_OF_YEAR',
    y= 'avg(VL_TOTAL)',
    markers= True,
    title= 'Média mensal do VL_TOTAL'
)
fig.update_layout(title= {'x': 0.5}, yaxis_title= 'Média do VL_TOTAL', xaxis_title= 'Mês')
fig.show()

##### Média trimestral

In [ ]:
avg_vl_total_per_quarter = vl_total_without_outliers.groupBy('QUARTER').agg(f.avg('VL_TOTAL')).orderBy('QUARTER').toPandas()

In [ ]:
fig = px.line(
    data_frame= avg_vl_total_per_quarter,
    x= 'QUARTER',
    y= 'avg(VL_TOTAL)',
    markers= True,
    title= 'Média trimestral do VL_TOTAL'
)
fig.update_layout(title= {'x': 0.5}, yaxis_title= 'Média do VL_TOTAL', xaxis_title= 'Trimestre')
fig.show()

**O último número do eixo x é referente ao trimestre**. Portanto, 20011 é o 1º trimestre do ano 2001. 

* Observação dos dados do 3º trimestre de 2003:

In [ ]:
df.where(df['YEAR'] == 2003).sort('VL_TOTAL', ascending= False).show(50)

Dos 50 maiores valores da variável `VL_TOTAL`, **a maior parte é do 3º bimestre de 2003**.

##### Média anual

In [ ]:
avg_vl_total_per_year = vl_total_without_outliers.groupBy('YEAR').agg(f.avg('VL_TOTAL')).orderBy('YEAR').toPandas()

In [ ]:
fig = px.line(
    data_frame= avg_vl_total_per_year,
    x= 'YEAR',
    y= 'avg(VL_TOTAL)',
    markers= True,
    title= 'Média anual do VL_TOTAL'
)
fig.update_layout(title= {'x': 0.5}, yaxis_title= 'Média do VL_TOTAL', xaxis_title= 'Ano')
fig.show()

## Coluna `VL_QUOTA`

### Estatísticas descritivas

In [ ]:
df.describe('VL_QUOTA').show(truncate= False)

### Média diária do `VL_QUOTA`

In [ ]:
avg_vl_quota_per_date = df.groupBy('DT_COMPTC').agg(f.avg('VL_QUOTA')).orderBy('DT_COMPTC').toPandas()

In [ ]:
fig = px.line(
    data_frame= avg_vl_quota_per_date,
    x= 'DT_COMPTC',
    y= 'avg(VL_QUOTA)',
    markers= True,
    title= 'Média diária da variável VL_QUOTA'
)
fig.update_layout(title= {'x': 0.5}, xaxis_title= 'Dia', yaxis_title= 'Média do VL_QUOTA')
fig.show()

### *Boxplot* da média diária do `VL_QUOTA`

In [ ]:
fig = px.box(
    data_frame= avg_vl_quota_per_date,
    x= 'avg(VL_QUOTA)', orientation= 'h',
    title= 'Boxplot da média diária do VL_QUOTA'
)
fig.update_layout(title= {'x':0.5}, yaxis_title= 'avg(VL_QUOTA)', xaxis_title= 'Valor')
fig.show()

### Remoção de *outliers* da variável `VL_QUOTA`

In [ ]:
Q1 = df.approxQuantile('VL_QUOTA', [0.25], 0.01)[0]
Q3 = df.approxQuantile('VL_QUOTA', [0.75], 0.01)[0]

In [ ]:
IIQ = Q3 - Q1

In [ ]:
inferior = Q1 - (1.5 * IIQ)
superior = Q3 + (1.5 * IIQ)

In [ ]:
vl_quota_without_outliers = df.where((df['VL_QUOTA'] >= inferior) & (df['VL_QUOTA'] <= superior) & (df['VL_QUOTA'] != 0))

In [ ]:
f'Número de linhas: {vl_quota_without_outliers.count()}'

### Análises após remoção dos *outliers*

#### Estatísticas descritivas

In [ ]:
vl_quota_without_outliers.describe('VL_QUOTA').show(truncate= False)

#### Medidas de tendência central do `VL_QUOTA`

##### Média diária

In [ ]:
avg_vl_quota_per_date = vl_quota_without_outliers.groupBy('DT_COMPTC').agg(f.avg('VL_QUOTA')).orderBy('DT_COMPTC').toPandas()

In [ ]:
fig = px.line(
    data_frame= avg_vl_quota_per_date,
    x= 'DT_COMPTC',
    y= 'avg(VL_QUOTA)',
    markers= True,
    title= 'Média diária do VL_QUOTA'
)
fig.update_layout(title= {'x': 0.5}, yaxis_title= 'Média do VL_QUOTA', xaxis_title= 'Dia')
fig.show()

##### Média mensal

In [ ]:
avg_vl_quota_per_month = vl_quota_without_outliers.groupBy('MONTH_OF_YEAR').agg(f.avg('VL_QUOTA')).orderBy('MONTH_OF_YEAR').toPandas()

In [ ]:
fig = px.line(
    data_frame= avg_vl_quota_per_month,
    x= 'MONTH_OF_YEAR',
    y= 'avg(VL_QUOTA)',
    markers= True,
    title= 'Média mensal do VL_QUOTA'
)
fig.update_layout(title= {'x': 0.5}, yaxis_title= 'Média do VL_QUOTA', xaxis_title= 'Mês')
fig.show()

##### Média trimestral

In [ ]:
avg_vl_quota_per_quater = vl_quota_without_outliers.groupBy('QUARTER').agg(f.avg('VL_QUOTA')).orderBy('QUARTER').toPandas()

In [ ]:
fig = px.line(
    data_frame= avg_vl_quota_per_quater,
    x= 'QUARTER',
    y= 'avg(VL_QUOTA)',
    markers= True,
    title= 'Média trimestral do VL_QUOTA'
)
fig.update_layout(title= {'x': 0.5}, yaxis_title= 'Média do VL_QUOTA', xaxis_title= 'Trimestre')

##### Média anual

In [ ]:
avg_vl_quota_per_year = vl_quota_without_outliers.groupBy('YEAR').agg(f.avg('VL_QUOTA')).orderBy('YEAR').toPandas()

In [ ]:
fig = px.line(
    data_frame= avg_vl_quota_per_year,
    x= 'YEAR',
    y= 'avg(VL_QUOTA)',
    markers= True,
    title= 'Média anual do VL_QUOTA'
)
fig.update_layout(title= {'x': 0.5}, yaxis_title= 'Média do VL_QUOTA', xaxis_title= 'Ano')
fig.show()

## Coluna `VL_PATRIM_LIQ`

### Estatísticas descritivas 

In [ ]:
df.describe('VL_PATRIM_LIQ').show(truncate= False)

### Média diária do `VL_PATRIM_LIQ`

In [ ]:
avg_vl_patrim_per_date = df.groupBy('DT_COMPTC').agg(f.avg('VL_PATRIM_LIQ')).orderBy('DT_COMPTC').toPandas()

In [ ]:
fig = px.line(
    data_frame= avg_vl_patrim_per_date,
    x= 'DT_COMPTC',
    y= 'avg(VL_PATRIM_LIQ)',
    markers= True,
    title= 'Média diária do VL_PATRIM_LIQ'
)
fig.update_layout(title= {'x': 0.5}, yaxis_title= 'Média do VL_PATRIM_LIQ', xaxis_title= 'Dia')
fig.show()

### *Boxplot* das médias diárias do `VL_PATRIM_LIQ`

In [ ]:
fig = px.box(
    data_frame= avg_vl_patrim_per_date,
    x= 'avg(VL_PATRIM_LIQ)', orientation= 'h',
    title= 'Boxplot das médias diárias do VL_PATRIM_LIQ'
)
fig.update_layout(title= {'x': 0.5}, yaxis_title= 'avg(VL_PATRIM_LIQ)', xaxis_title= 'Valor')
fig.show()

### Remoção de *outliers* da variável `VL_PATRIM_LIQ`

In [ ]:
Q1 = df.approxQuantile('VL_PATRIM_LIQ', [0.25], 0.01)[0]
Q3 = df.approxQuantile('VL_PATRIM_LIQ', [0.75], 0.01)[0]

In [ ]:
IIQ = Q3 - Q1

In [ ]:
inferior = Q1 - (1.5 * IIQ)
superior = Q3 + (1.5 * IIQ)

In [ ]:
vl_patrim_without_outliers = df.where((df['VL_PATRIM_LIQ'] >= inferior) & (df['VL_PATRIM_LIQ'] <= superior))

### Análises após a remoção dos *outliers*

#### Estatísticas descritivas

In [ ]:
vl_patrim_without_outliers.describe('VL_PATRIM_LIQ').show(truncate= False)

#### Medidas de tendência central `VL_PATRIM_LIQ`

##### Média mensal

In [133]:
avg_vl_patrim_per_month = vl_patrim_without_outliers.groupBy('MONTH_OF_YEAR').agg(f.avg('VL_PATRIM_LIQ')).orderBy('MONTH_OF_YEAR').toPandas()

In [169]:
fig = plt.line(
    data_frame= avg_vl_patrim_per_month,
    x= 'MONTH_OF_YEAR',
    y= 'avg(VL_PATRIM_LIQ)',
    markers= True,
    title= 'Médias mensais do VL_PATRIM_LIQ entre 2000 e 2023'
)
fig.update_layout(title= {'x': 0.5}, yaxis_title= 'Média do VL_PATRIM_LIQ', xaxis_title= 'Mês')
fig.show()

#adnaodnao

AttributeError: 'function' object has no attribute 'line'

##### Média trimestral

In [135]:
avg_vl_patrim_per_quarter = vl_patrim_without_outliers.groupBy('QUARTER').agg(f.avg('VL_PATRIM_LIQ')).orderBy('QUARTER').toPandas()

In [136]:
fig = px.line(
    data_frame= avg_vl_patrim_per_quarter,
    x= 'QUARTER',
    y= 'avg(VL_PATRIM_LIQ)',
    markers= True,
    title= 'Médias trimestrais do VL_PATRIM_LIQ entre 2000 e 2023',
    color_discrete_sequence= ['red']
)
fig.update_layout(title= {'x': 0.5}, yaxis_title= 'Média do VL_PATRIM_LIQ', xaxis_title= 'Trimestre')
fig.show()

##### Média anual

In [137]:
avg_vl_patrim_per_year = vl_patrim_without_outliers.groupBy('YEAR').agg(f.avg('VL_PATRIM_LIQ')).orderBy('YEAR').toPandas()

In [138]:
fig = px.line(
    data_frame= avg_vl_patrim_per_year,
    x= 'YEAR',
    y= 'avg(VL_PATRIM_LIQ)',
    markers= True,
    color_discrete_sequence= ['green'],
    title= 'Médias anuais do VL_PATRIM_LIQ entre 2000 e 2023'
)
fig.update_layout(title= {'x': 0.5}, yaxis_title= 'Média do VL_PATRIM_LIQ', xaxis_title= 'Ano')
fig.show()

## Coluna `CAPTC_DIA`

### Estatísticas descritivas 

In [139]:
df.describe('CAPTC_DIA').show(truncate= False)

+-------+-------------------+
|summary|CAPTC_DIA          |
+-------+-------------------+
|count  |57626454           |
|mean   |2196028.455280401  |
|stddev |1.992992103041894E8|
|min    |-828000.0          |
|max    |8.3226001E11       |
+-------+-------------------+



### Média diária da `CAPTC_DIA`

In [140]:
avg_captc_per_date = df.groupBy('DT_COMPTC').agg(f.avg('CAPTC_DIA')).orderBy('DT_COMPTC').toPandas()

C:\Spark\spark-3.3.1-bin-hadoop3\python\pyspark\sql\pandas\conversion.py:248: FutureWarning:

Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead



In [141]:
fig = px.line(
    data_frame= avg_captc_per_date,
    x= 'DT_COMPTC',
    y= 'avg(CAPTC_DIA)',
    markers= True,
    title= 'Médias diárias da CAPTC_DIA entre 2000 e 2023'
)
fig.show()

### *Boxplot* da média diária da `CAPTC_DIA`

In [142]:
fig = px.box(
    data_frame= avg_captc_per_date,
    x= 'avg(CAPTC_DIA)', orientation= 'h',
    title= 'Boxplot das médias diárias da CAPTC_DIA'
)
fig.update_layout(title= {'x': 0.5}, yaxis_title= 'CAPTC_DIA', xaxis_title= 'Valor')
fig.show()

### Remoção de *outliers* da variável `CAPTC_DIA`

In [143]:
captc_Q1 = df.approxQuantile('CAPTC_DIA', [0.25], 0.01)[0]
captc_Q3 = df.approxQuantile('CAPTC_DIA', [0.75], 0.01)[0]

Para variável `CAPTC_DIA` verifica-se que o 1º e o 3º quartil possuem, respectivamente, os valores:

In [144]:
print(f'Q1: {captc_Q1}')
print(f'Q3: {captc_Q3}')

Q1: 0.0
Q3: 0.0


O 3º quatil (Q3) mostra que pelo menos 75% dos dados na variável `CAPTC_DIA` equivalem a **ZERO**. 

In [145]:
captc_equal_zero = df.where(df['CAPTC_DIA'] == 0).count() # quantidade de dados em CAPTC_DIA com valor zero.
total_rows = df.count() # número de linhas, ou seja, número de dados.

In [146]:
captc_equal_zero_percent = round(((captc_equal_zero/total_rows) * 100), 2)

In [147]:
print(f'Quantidade de dados de valor ZERO: {captc_equal_zero}.')
print(f'O que equivale a {captc_equal_zero_percent}% dos dados na variável.')

Quantidade de dados de valor ZERO: 45873267.
O que equivale a 79.6% dos dados na variável.


* Dado discrepante no gráfico das médias diárias da `CAPTC_DIA`:

No gráfico anterior das médias diárias da variável há um pico na ordem de bilhão.

In [148]:
max_captc_per_date = avg_captc_per_date['avg(CAPTC_DIA)'].max()
print(f'Com valor: R$ {max_captc_per_date}')

Com valor: R$ 1127752249.3224933


Tal dado é do dia mostrado na tabela abaixo:

In [149]:
avg_captc_per_date.loc[avg_captc_per_date['avg(CAPTC_DIA)'] == max_captc_per_date]

,DT_COMPTC,avg(CAPTC_DIA)
799,2003-01-07,1.127752e+09


Ao buscar os dados do dia 01 de janeiro de 2003 e ordenando-os, verifica-se que:

In [150]:
df.where(df['DT_COMPTC'] == '2003-01-07').sort('CAPTC_DIA', ascending= False).show(10)

+------+---------+------------------+-------------------+--------+---------+-------------+------------+---------+--------+-----+----+-------------+-------+
| index| TP_FUNDO|        CNPJ_FUNDO|          DT_COMPTC|VL_TOTAL| VL_QUOTA|VL_PATRIM_LIQ|   CAPTC_DIA| RESG_DIA|NR_COTST|MONTH|YEAR|MONTH_OF_YEAR|QUARTER|
+------+---------+------------------+-------------------+--------+---------+-------------+------------+---------+--------+-----+----+-------------+-------+
|323979|    FITVM|88.007.505/0001-29|2003-01-07 00:00:00|    null|1.3367638|    6333000.0|8.3226001E11|  13000.0|    null|    1|2003|      2003-01|  20031|
|299443|    FITVM|05.371.423/0001-31|2003-01-07 00:00:00|    null|1.1376755|     1.3157E7|   5000000.0|5000000.0|    null|    1|2003|      2003-01|  20031|
|271772|    FITVM|04.806.420/0001-10|2003-01-07 00:00:00|    null|2.8258568|     3.4671E7|   1494000.0|      0.0|    null|    1|2003|      2003-01|  20031|
|225590|    FITVM|03.707.396/0001-08|2003-01-07 00:00:00|    nul

O dado com index 323979 tem `CAPTC_DIA` de R$ 8,32 E10¹¹.

Valor muito acima da média da variável que é de aproximadamente R$ 2,2 milhões.


O dado discrepante é, portanto, retirado da variável `CAPTC_DIA` para realização de análises:

In [151]:
captc_without_outliers = df.where(df['index'] != 323979)

In [152]:
captc_without_outliers.where(df['index'] == 323979).show()

+-----+--------+----------+---------+--------+--------+-------------+---------+--------+--------+-----+----+-------------+-------+
|index|TP_FUNDO|CNPJ_FUNDO|DT_COMPTC|VL_TOTAL|VL_QUOTA|VL_PATRIM_LIQ|CAPTC_DIA|RESG_DIA|NR_COTST|MONTH|YEAR|MONTH_OF_YEAR|QUARTER|
+-----+--------+----------+---------+--------+--------+-------------+---------+--------+--------+-----+----+-------------+-------+
+-----+--------+----------+---------+--------+--------+-------------+---------+--------+--------+-----+----+-------------+-------+



### Análises após a verificação de existência dos *outliers*

#### Medidas de tendência central `CAPTC_DIA`

##### Média mensal

In [153]:
avg_captc_per_month = captc_without_outliers.groupBy('MONTH_OF_YEAR').agg(f.avg('CAPTC_DIA')).orderBy('MONTH_OF_YEAR').toPandas()

In [154]:
fig = px.line(
    data_frame= avg_captc_per_month,
    x= 'MONTH_OF_YEAR',
    y= 'avg(CAPTC_DIA)',
    markers= True,
    title= 'Médias mensais da CAPTC_DIA entre 2000 e 2023'
)
fig.update_layout(title= {'x': 0.5}, yaxis_title= 'Média da CAPTC_DIA', xaxis_title= 'Mês')
fig.show()

In [155]:
fig = px.line(
    data_frame= avg_captc_per_month,
    x= 'MONTH_OF_YEAR',
    y= 'avg(CAPTC_DIA)',
    markers= True,
    title= 'Médias mensais da CAPTC_DIA entre 2000 e 2023' 
)
fig.update_layout(title= {'x': 0.5}, yaxis_title= 'Média da CAPTC_DIA', xaxis_title= 'Mês')
fig.show()

##### Média trimestral

In [156]:
avg_captc_per_quarter = captc_without_outliers.groupBy('QUARTER').agg(f.avg('CAPTC_DIA')).orderBy('QUARTER').toPandas()

In [157]:
avg_captc_per_quarter

,QUARTER,avg(CAPTC_DIA)
0,20001,1.944444e+04
1,20002,9.123077e+03
2,20003,2.525758e+04
3,20004,7.580645e+03
4,20011,5.343750e+03
...,...,...
88,20221,2.922184e+06
89,20222,2.926555e+06
90,20223,2.593686e+06
91,20224,2.466764e+06


## Coluna `RESG_DIA`

### Estatísticas descritivas

In [158]:
df.describe('RESG_DIA').show(truncate= True)

+-------+--------------------+
|summary|            RESG_DIA|
+-------+--------------------+
|  count|            57626454|
|   mean|   2112156.705014613|
| stddev|6.0093168497690335E7|
|    min|           -796000.0|
|    max|  1.1114476784541E11|
+-------+--------------------+



### Média diária do `RESG_DIA`

In [159]:
avg_resg_per_date = df.groupBy('DT_COMPTC').agg(f.avg('RESG_DIA')).orderBy('DT_COMPTC').toPandas()

C:\Spark\spark-3.3.1-bin-hadoop3\python\pyspark\sql\pandas\conversion.py:248: FutureWarning:

Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead



In [160]:
fig = px.line(
    data_frame= avg_resg_per_date,
    x= 'DT_COMPTC',
    y= 'avg(RESG_DIA)',
    markers= True,
    title= 'Médias diárias do RESG_DIA entre 2000 e 2023'
)
fig.update_layout(title= {'x': 0.5}, yaxis_title= 'Média do RESG_DIA', xaxis_title= 'Dia')
fig.show()

### *Boxplot* das médias diárias do `RESG_DIA`

In [161]:
fig = px.box(
    data_frame= avg_resg_per_date,
    x= 'avg(RESG_DIA)', orientation= 'h',
    title= 'Boxplot das médias diáris do RESG_DIA entre 2000 e 2023' 
)
fig.update_layout(title= {'x': 0.5}, yaxis_title= 'RESG_DIA', xaxis_title= 'Valor')
fig.show()

In [162]:
df.where(df['RESG_DIA'] == 0).count()

44635357

In [163]:
df.filter(df['RESG_DIA'] != 0).count()

12991097

### Remoção de *outliers* do `RESG_DIA`

In [164]:
resg_Q1 = df.approxQuantile('RESG_DIA', [0.25], 0.01)[0]
resg_Q3 = df.approxQuantile('RESG_DIA', [0.75], 0.01)[0]

In [165]:
resg_Q1

0.0

In [166]:
resg_Q3

0.0

In [167]:
resg_IIQ = resg_Q3 - resg_Q1 

In [168]:
df.columns

['index',
 'TP_FUNDO',
 'CNPJ_FUNDO',
 'DT_COMPTC',
 'VL_TOTAL',
 'VL_QUOTA',
 'VL_PATRIM_LIQ',
 'CAPTC_DIA',
 'RESG_DIA',
 'NR_COTST',
 'MONTH',
 'YEAR',
 'MONTH_OF_YEAR',
 'QUARTER']